In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import gc
import random
import re
from sklearn.metrics import *
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
base_path = '../preprocessed data/'

test_err = pd.read_csv(base_path + 'new_test_err.csv', parse_dates=['time'])
test_quality = pd.read_csv(base_path + 'new_test_quality.csv')
train_8 = pd.read_csv('../training set/train_case3_8.csv')
train_9 = pd.read_csv('../training set/train_case3_9.csv')

# feature engineering

In [3]:
df1 = test_err.copy()
df1

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,2020-11-01 03:02:27,model_1,04.16.3553,31,1
1,30000,2020-11-01 03:02:27,model_1,04.16.3553,33,2
2,30000,2020-11-01 03:02:28,model_1,04.16.3553,15,1
3,30000,2020-11-01 03:02:56,model_1,04.16.3553,22,1
4,30000,2020-11-01 03:03:00,model_1,04.16.3553,11,1
...,...,...,...,...,...,...
15527216,44998,2020-11-30 21:00:50,model_1,04.16.3553,40,0
15527217,44998,2020-11-30 21:18:31,model_1,04.16.3553,31,1
15527218,44998,2020-11-30 21:18:32,model_1,04.16.3553,15,1
15527219,44998,2020-11-30 21:22:59,model_1,04.16.3553,16,1


In [4]:
# counts column 추가
df1['counts'] = 1
df1

,user_id,time,model_nm,fwver,errtype,errcode,counts
0,30000,2020-11-01 03:02:27,model_1,04.16.3553,31,1,1
1,30000,2020-11-01 03:02:27,model_1,04.16.3553,33,2,1
2,30000,2020-11-01 03:02:28,model_1,04.16.3553,15,1,1
3,30000,2020-11-01 03:02:56,model_1,04.16.3553,22,1,1
4,30000,2020-11-01 03:03:00,model_1,04.16.3553,11,1,1
...,...,...,...,...,...,...,...
15527216,44998,2020-11-30 21:00:50,model_1,04.16.3553,40,0,1
15527217,44998,2020-11-30 21:18:31,model_1,04.16.3553,31,1,1
15527218,44998,2020-11-30 21:18:32,model_1,04.16.3553,15,1,1
15527219,44998,2020-11-30 21:22:59,model_1,04.16.3553,16,1,1


In [5]:
# multiple_model
model_df =  test_err[['user_id', 'model_nm']]
model_df = model_df.drop_duplicates(subset=None, keep='first')

model_num = model_df.user_id.value_counts(sort=False)
user_has_multiple_models = model_num > 1

user_has_multiple_models = user_has_multiple_models.astype(int)
user_has_multiple_models = user_has_multiple_models.to_frame(name='multiple_model')
user_has_multiple_models.head(7)

,multiple_model
30000,1
30001,0
30002,0
30003,0
30004,1
30005,0
30006,0


In [6]:
# period
test_err['day'] = test_err.time.dt.day
test_err['hour'] = test_err.time.dt.hour

user_day_df = test_err.groupby('user_id')['day'].unique().to_frame().reset_index()
user_day_df['period'] = 0 

for i in range(len(user_day_df.index)):
   user_day_df['period'][i] = len(user_day_df['day'][i])

user_day_df

,user_id,day,period
0,30000,"[1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",29
1,30001,"[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 1...",28
2,30002,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
3,30003,"[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 1...",28
4,30004,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
...,...,...,...
14993,44994,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
14994,44995,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
14995,44996,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",30
14996,44997,"[1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...",28


In [7]:
user_day_df.index = user_day_df['user_id']
user_day_df.drop(columns=['user_id', 'day'], inplace=True)

user_day_df

,period
user_id,
30000,29
30001,28
30002,30
30003,28
30004,30
...,...
44994,30
44995,30
44996,30


In [8]:
# 1. one-hot-encoding
df1.drop(columns=['time', 'errcode'], inplace=True)
df1 = pd.get_dummies(df1, columns=['model_nm', 'fwver', 'errtype'])
df1

,user_id,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,...,errtype_33,errtype_34,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42
0,30000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30000,1,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,30000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,30000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,30000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15527216,44998,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
15527217,44998,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15527218,44998,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15527219,44998,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# 모든 column 합쳐주기
df1 = df1.groupby('user_id').sum()
dfs = [df1, user_has_multiple_models, user_day_df]
dfs = dfs[0].join(dfs[1:])
dfs

,counts,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,errtype_35,errtype_36,errtype_37,errtype_38,errtype_39,errtype_40,errtype_41,errtype_42,multiple_model,period
user_id,,,,,,,,,,,,,,,,,,,,,
30000,2731,0.0,2669.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,3.0,0.0,2.0,92.0,0.0,2.0,1,29
30001,284,0.0,0.0,0.0,284.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,28
30002,903,903.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,14.0,113.0,1.0,2.0,0,30
30003,328,328.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,1.0,1.0,0.0,0.0,40.0,8.0,0.0,0,28
30004,871,0.0,622.0,249.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,48.0,90.0,0.0,1,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44994,1115,0.0,1115.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,100.0,87.0,0.0,0,30
44995,515,515.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,12.0,17.0,0.0,0,30
44996,1915,1915.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,329.0,2.0,3.0,0,30


In [10]:
test_c = list(dfs.columns)
train_8_c = list(train_8.columns)
train_9_c = list(train_9.columns)

len(test_c), len(train_8_c), len(train_9_c)

(93, 69, 71)

In [11]:
for i in train_8_c:
    if i not in test_c:
        print(i)

fwver_04.16.2641
fwver_04.16.3345
fwver_04.22.1442
fwver_04.33.1095
fwver_05.15.2090
fwver_05.15.2122


In [12]:
for i in train_9_c:
    if i not in test_c:
        print(i)

fwver_04.16.2641
fwver_04.16.3345
fwver_04.22.1442
fwver_04.33.1095
fwver_05.15.2090
fwver_05.15.2122


In [13]:
# # test에는 없는 fwver 삭제
train_8.drop(columns=['fwver_04.16.2641', 'fwver_04.16.3345', 'fwver_04.22.1442', 
                    'fwver_04.33.1095', 'fwver_05.15.2090', 'fwver_05.15.2122'], inplace=True)

train_9.drop(columns=['fwver_04.16.2641', 'fwver_04.16.3345', 'fwver_04.22.1442', 
                    'fwver_04.33.1095', 'fwver_05.15.2090', 'fwver_05.15.2122'], inplace=True)

In [14]:
d_columns = []
for i in test_c:
    if i not in train_8_c:
        d_columns.append(i)
d_columns

['fwver_03.11.1167',
 'fwver_04.16.3553',
 'fwver_04.22.1170',
 'fwver_04.22.1448',
 'fwver_04.22.1478',
 'fwver_04.22.1608',
 'fwver_04.22.1750',
 'fwver_04.22.1772',
 'fwver_04.33.1185',
 'fwver_04.33.1261',
 'fwver_04.73.2237',
 'fwver_04.73.2569',
 'fwver_04.73.2577',
 'fwver_04.82.1684',
 'fwver_04.82.1778',
 'fwver_05.15.2138',
 'fwver_05.66.3237',
 'fwver_10',
 'fwver_10.22.1770',
 'fwver_10.22.1780',
 'errtype_4',
 'errtype_7',
 'errtype_11',
 'errtype_12',
 'errtype_16',
 'errtype_21',
 'errtype_23',
 'errtype_27',
 'errtype_28',
 'errtype_37']

In [15]:
test_8 = dfs.drop(columns=d_columns)
len(test_8.columns), len(train_8.columns)

(63, 63)

In [16]:
d_columns = []
for i in test_c:
    if i not in train_9_c:
        d_columns.append(i)
d_columns

['fwver_03.11.1167',
 'fwver_04.16.3553',
 'fwver_04.22.1170',
 'fwver_04.22.1448',
 'fwver_04.22.1478',
 'fwver_04.22.1608',
 'fwver_04.22.1750',
 'fwver_04.22.1772',
 'fwver_04.33.1261',
 'fwver_04.73.2237',
 'fwver_04.73.2569',
 'fwver_04.73.2577',
 'fwver_04.82.1684',
 'fwver_05.15.2138',
 'fwver_05.66.3237',
 'fwver_10',
 'fwver_10.22.1770',
 'fwver_10.22.1780',
 'errtype_4',
 'errtype_7',
 'errtype_11',
 'errtype_12',
 'errtype_16',
 'errtype_21',
 'errtype_23',
 'errtype_27',
 'errtype_28',
 'errtype_37']

In [17]:
test_9 = dfs.drop(columns=d_columns)
len(test_9.columns), len(train_9.columns)

(65, 65)

In [19]:
test_8.to_csv('../training set/test_case3_8.csv', index=False)
test_9.to_csv('../training set/test_case3_9.csv', index=False)
train_8.to_csv('../training set/train_case3_8.csv', index=False)
train_8.to_csv('../training set/train_case3_9.csv', index=False)